In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalMaxPooling2D
import os

train_dir = "pothole_image_data/train"
test_dir = "pothole_image_data/test"

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary"
)
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary"
)

# Load pre-trained model
input_shape = (150, 150, 3)
pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

# Freeze layers
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

# Unfreeze remaining layers
for layer in pre_trained_model.layers[15:]:
    layer.trainable = True

# Add custom layers
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # For binary classification

model = Model(pre_trained_model.input, x)

# Compile model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator
)

# Save model
os.makedirs("D:/kclas_projects/pothole", exist_ok=True)
model.save("D:/kclas_projects/pothole/model.h5")


Found 319 images belonging to 2 classes.
Found 299 images belonging to 2 classes.
16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9595 - loss: 0.0985 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
